# 12: Predictions & Alerts

Generate predictions on new data and produce alerts with bench recommendations.

## Purpose

1. **Load** trained Random Forest model and thresholds
2. **Predict** severity (0-5) on new IMU data
3. **Generate** alerts based on severity mapping
4. **Output** actionable bench recommendations

## Alert Action Items

- **None (0-1)**: Continue training
- **Low (2)**: Monitor athlete closely
- **Medium (3)**: Consider reduced training
- **High (4)**: Recommend bench / medical evaluation
- **Critical (5)**: Immediate intervention required


In [ ]:
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path
import sys
sys.path.append('../../src')

from sledhead_imu.models.random_forest import predict_random_forest
from sledhead_imu.alerts import generate_alerts_from_severity, summarize_alerts, get_critical_alerts

# Setup paths
data_dir = Path('../data')
models_dir = data_dir / '10_models'
validate_dir = data_dir / '11_metrics_validate_cutoffs'
predictions_dir = data_dir / '12_predictions_alerts'

# Load model
model_file = models_dir / 'rf' / 'model.pkl'
if model_file.exists():
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    print("✓ Loaded trained Random Forest model")
else:
    print("⚠️  Model not found. Run 10_train_random_forest.ipynb first.")
    model = None

# Load severity mapping
thresholds_file = validate_dir / 'thresholds' / 'severity_mapping.json'
if thresholds_file.exists():
    with open(thresholds_file, 'r') as f:
        severity_mapping = json.load(f)
    print("✓ Loaded severity-to-alert mapping")
else:
    print("⚠️  Thresholds not found. Run 11_validate_define_cutoffs.ipynb first.")
    severity_mapping = None


In [ ]:
# Load new data for prediction
# In production, this would be new IMU data processed through the pipeline

if model is not None:
    # For demo, use test set as "new" data
    test_X_file = data_dir / '09_splits' / 'test' / 'X_test.csv'
    
    if test_X_file.exists():
        X_new = pd.read_csv(test_X_file)
        
        # Generate predictions
        severity_predictions = predict_random_forest(model, X_new)
        
        print(f"✓ Generated predictions for {len(X_new)} samples")
        print(f"\nPredicted severity distribution:")
        severity_counts = pd.Series(severity_predictions).value_counts().sort_index()
        for sev, count in severity_counts.items():
            print(f"  Severity {sev}: {count} samples")
    else:
        print("⚠️  Test data not found. Skipping predictions.")
        severity_predictions = None
else:
    print("⚠️  Model not available. Skipping predictions.")
    severity_predictions = None


In [ ]:
# Generate alerts from predictions
if severity_predictions is not None and severity_mapping is not None:
    # Create athlete IDs for demo
    athlete_ids = [f"A{i:04d}" for i in range(len(severity_predictions))]
    
    # Generate alerts
    alerts_df = generate_alerts_from_severity(
        severity_predictions, 
        athlete_ids, 
        severity_mapping
    )
    
    print("✓ Generated alerts")
    print(f"\nAlert summary:")
    alert_summary = summarize_alerts(alerts_df)
    for level, count in sorted(alert_summary.items()):
        print(f"  {level.capitalize()}: {count} athletes")
    
    # Show sample alerts
    print(f"\nSample alerts (first 5 rows):")
    print(alerts_df[['athlete_id', 'severity', 'alert_level']].head().to_string(index=False))
    
else:
    print("⚠️  Skipping alert generation - missing predictions or thresholds")
    alerts_df = None


In [ ]:
# Show critical alerts requiring immediate attention
if alerts_df is not None:
    critical = get_critical_alerts(alerts_df)
    
    if len(critical) > 0:
        print("⚠️  CRITICAL ALERTS - IMMEDIATE ATTENTION REQUIRED")
        print("="*80)
        print(critical[['athlete_id', 'severity', 'alert_level']].to_string(index=False))
        print(f"\nAction: {len(critical)} athlete(s) require bench recommendation or medical evaluation")
    else:
        print("✓ No critical alerts")
    
    # Save outputs
    predictions_dir.mkdir(parents=True, exist_ok=True)
    outputs_dir = predictions_dir / 'outputs'
    outputs_dir.mkdir(exist_ok=True)
    
    # Save all alerts
    if alerts_df is not None:
        alerts_df.to_csv(outputs_dir / 'all_alerts.csv', index=False)
        print(f"\n✓ Saved all alerts to {outputs_dir / 'all_alerts.csv'}")
        
        # Save critical alerts
        if len(critical) > 0:
            critical.to_csv(outputs_dir / 'critical_alerts.csv', index=False)
            print(f"✓ Saved critical alerts to {outputs_dir / 'critical_alerts.csv'}")
    
    print("\n✅ Predictions and alerts complete!")
else:
    print("⚠️  No alerts generated")
